Import neccessary packages

In [1]:
import numpy as np
import tensorflow as tf
import pickle
import cv2
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
# from tf.keras.layers import BatchNormalization
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D,Conv3D
from keras.layers.convolutional import MaxPooling2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'keras.layers.normalization'

In [2]:
EPOCHS = 50
INIT_LR = 1e-3 #0.0003
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = '../input/plantvillage/'
width=256
height=256
depth=3

Function to convert images to array

In [3]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

Fetch images from directory

In [4]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    print(root_dir)
    
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)
        
    
    for plant_folder in root_dir :
        print(plant_folder)
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        plant_disease_folder_list.remove("Potato___Early_blight")
        plant_disease_folder_list.remove("Potato___Late_blight")
        plant_disease_folder_list.remove("Potato___healthy")
        plant_disease_folder_list.remove("Tomato_Early_blight")
        
        plant_disease_folder_list.remove("Tomato_Late_blight")
        plant_disease_folder_list.remove("Pepper__bell___healthy")
        plant_disease_folder_list.remove("Tomato__Tomato_mosaic_virus")
        
        plant_disease_folder_list.remove("Tomato_Leaf_Mold")
        plant_disease_folder_list.remove("Tomato_Spider_mites_Two_spotted_spider_mite")
        plant_disease_folder_list.remove("Tomato__Tomato_YellowLeaf__Curl_Virus")
        plant_disease_folder_list.remove("Pepper__bell___Bacterial_spot")
        
        print("removed")
        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:200]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
['PlantVillage']
PlantVillage
removed
[INFO] Processing Tomato_Bacterial_spot ...
[INFO] Processing Tomato_Septoria_leaf_spot ...
[INFO] Processing Tomato_healthy ...
[INFO] Processing Tomato__Target_Spot ...
[INFO] Image loading completed


Get Size of Processed Image

In [5]:
image_size = len(image_list)

Transform Image Labels uisng [Scikit Learn](http://scikit-learn.org/)'s LabelBinarizer

In [6]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

Print the classes

In [7]:
print(label_binarizer.classes_)

['Tomato_Bacterial_spot' 'Tomato_Septoria_leaf_spot' 'Tomato__Target_Spot'
 'Tomato_healthy']


In [8]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

In [9]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42)

[INFO] Spliting data to train, test


In [10]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [11]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(16, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(8))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

Model Summary

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 85, 85, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 85, 85, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 85, 85, 64)        0         
__________

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

[INFO] training network...


In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

Epoch 1/50
20/20 [==============================] - 96s 5s/step - loss: 1.1990 - acc: 0.4906 - val_loss: 1.1649 - val_acc: 0.5687
Epoch 2/50
20/20 [==============================] - 95s 5s/step - loss: 0.8493 - acc: 0.6687 - val_loss: 1.1731 - val_acc: 0.4000
Epoch 3/50
20/20 [==============================] - 95s 5s/step - loss: 0.7884 - acc: 0.7094 - val_loss: 1.4091 - val_acc: 0.4562
Epoch 4/50
20/20 [==============================] - 95s 5s/step - loss: 0.7413 - acc: 0.7375 - val_loss: 1.1620 - val_acc: 0.5062
Epoch 5/50
20/20 [==============================] - 95s 5s/step - loss: 0.6286 - acc: 0.8062 - val_loss: 0.8418 - val_acc: 0.6687
Epoch 6/50
20/20 [==============================] - 94s 5s/step - loss: 0.5300 - acc: 0.8422 - val_loss: 0.6814 - val_acc: 0.7188
Epoch 7/50
20/20 [==============================] - 94s 5s/step - loss: 0.4631 - acc: 0.8781 - val_loss: 0.8150 - val_acc: 0.6438
Epoch 8/50
20/20 [==============================] - 94s 5s/step - loss: 0.3989 - acc: 0.90

Plot the train and val curve

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

Model Accuracy

In [ ]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

Save model using Pickle

In [ ]:
# save the model to disk
print("[INFO] Saving model...")
pickle.dump(model,open('cnn_model.pkl', 'wb'))

In [ ]:
model.save("leaf.h5")

In [ ]:

leaf = ['Pepper__bell___Bacterial_spot' ,'Pepper__bell___healthy',
 'Potato___Early_blight', 'Potato___Late_blight' ,'Potato___healthy',
 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight',
 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot',
 'Tomato_Spider_mites_Two_spotted_spider_mite' ,'Tomato__Target_Spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus',
 'Tomato_healthy']

  



In [ ]:
type(x_test[0])
res = model.predict(x_test)
for d in res:
    print(np.argmax(d))

In [ ]:
import os
os.chdir(r'/kaggle/working')

In [ ]:
from IPython.display import FileLink
FileLink(r'leaf.h5')

Matrix
lib
epoch
training
testing
algo
mo

